For better readability, you may visit the html rendered version on my [github page](https://matthieuvion.github.io/wzkd/cod_api_doc.html)

# COD private API documentation through CallofDuty.py

[Credits to github.com/EthanC](https://github.com/EthanC/CallofDuty.py) <br>For this well made Python client <br><br>
> *"CallofDuty.py is an asynchronous, object-oriented Python wrapper for the Call of Duty API."*

---

#### Context & objectives

In this notebook, we will test & learn how to use this **--non official, Call of Duty (COD) client** to access players' stats, with a strong inclination towards **Warzone**. <br>
If you didn't know, *Warzone* is a free Battle Royale game, part of the Call of Duty universe, developped by Activision and is --said to be, played by 100 M people.<br> 
Players (Warzone or the more classic online multiplayers modes of the different COD games) have access to some stats (score, kills/deaths ratio, rank...) on [my.callofduty.com](https://my.callofduty.com/) but they're poorly put together.<br> 
Cause or consequence, this led to to the creation of a rich ecosytem of --often very popular, websites (codtracker.gg, wzranked...) promising progression trackers and deeper analysis to players. <br>

As Activision is using a **"private" API** (with no support) on the callofduty.com website and the **documentation** is very **sparse** otherwise, **this notebook intends to** : 
- Progress back and forth between both client and COD API to get an overall picture of what does what.
- Present a quick glance at client's architecture as well are more details about available methods & their outputs. 
- Detail **additional methods** to test another COD API route that is listed on Postman, or remove some filters that may prevent us from fully exploiting the API.
- Provide --though being very, **very far from exhaustive**, a deeper layer of explanation (on top of ressources listed below) on how COD API operates (Warzone oriented).

#### Resources

AFAIK, the most complete wrappers for COD API are the one used here and another written in NodeJS [(Github)](https://docs.codapi.dev/getting-started). <br>
A good starting point if you want to get your hands dirty in COD stats would be to read both code. Besides, I would also recommend that you read the [documentation](https://docs.codapi.dev/getting-started) of the NodeJS wrapper as well as this [Postman tool](https://documenter.getpostman.com/view/5519582/SzzgAefq).<br>They will give you a good overall idea of which endpoints, authentification and data are at your disposal.

#### Install and run

Personal preferences here but I'm using miniconda (conda) as a environment manager (could be pyenv etc.) and Poetry for dependency managing and packaging. <br>
In my conda environment I have Python 3.9 (needed for the client), Jupyter and Poetry installed.
1. Create a new project with `poetry new your_project_name` or, if you have already a pre-populated directory, `cd your_existing_project` and then `poetry init`. Cf. [Poetry documentation](https://python-poetry.org/docs/)
2. Install the Call of Duty client : `poetry add callofduty.py`. Poetry will make sure to install all the requirements.
3. Run this notebook : `poetry run jupyter lab`, or `poetry shell` to start a new shell and then `jupyter lab` in the newly opened terminal.
This will ensure you have access to all dependencies, in a custom and clean environment, thus callofduty.py and the notebook perfectly

In [1]:
import asyncio
import os
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone


import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType

#### Login to the API, start client session

Two ways to authenticate to COD API. Once you're logged in, you will have access to either private (your info) or protected routes that may supply data for any given user. [Postman](https://docs.codapi.dev/getting-started) to further know what's happening under the hood. <br> 1. Login & password with `callofduty.Login(activision_email, pwd)`. I think it doesn't work anymore since Activision added a reCaptcha (but seems to be doable with the [NodeJS package](https://docs.codapi.dev/getting-started) that's using puppeteers + a plugin to bypass it).<br>2. Single Sign On (sso) `callofduty.Login(SSO_TOKEN)<p> added recently, that uses a SSO token you get while logging to Activision through your platform of choice (Bnet, Xbox, PS).

In [2]:
# Using SSO
# We're storing our SSO token in an .env file stored locally to separate our config from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

#### COD universe, endpoints & client

COD ecosystem is indeed diverse :
- You can have access to one or multiple titles (*Modern Warfare*, *Black Ops Cold War* ...)
- playable locally or more likely online multiplayer
- within every game, several 'modes', e.g the the 'Battle Royale' *Warzone*, also with different maps (called 'modes' also :p)
- through multiple platforms, depending on the game (Steam, Battle Net, Xbox Live...)

Players need to have enabled their visibility to 'on' (off by default, since January 2021) in their settings so their profile is searchable.<br>
The way the Activision API works is that you generally need to specify, for any given player's, its gamertag associated to a given platform and then the title/mode/sub you want to get data from (a player can have two different gamertags whether he is using BattleNet or Playstation Live).<br>
For in-depth access to player's stats, one generally needs to specify the Platform (e.g. Activision), Title (e.g. Modern Warfare) and the Mode (e.g. multiplayer)<br>
Since it's our focus here, once we identified a player by his gamertag & associated platform, we will usually specify `title = modernwarfare` and `mode = warzone` as Warzone is a free mode developped within the Modern Warfare engine and thus organized this way in the API.<br>
Luckily for us the python wrapper handles the naming in an enums.py file (`Mode, Platform...`) to build the endpoints smoothly, as well as objects/classes (`client.py, match.py, player.py,...`) to work with.

Example of a GET request built in the client to access the API : <br>
Cf. [Postman](https://docs.codapi.dev/getting-started) for details about API's versions & path variables as well as differences between between private, protected and public routes

> `Request("GET",f"api/papi-client/leaderboards/v2/title/{title}/platform/{platform}/time/{timeFrame}/type/{gameType}/mode/{gameMode}/page/{page}",)`

#### Client architecture & table of (some) useful methods

The table below is quite complete but not exhaustive, took long enough to do ^_^ <br>
Mainly a good way to have the big picture on protected/public routes, useful to gather player stats.<br>
As mentioned on Postman, routes are either private, public or protected; this will be our plan when we explore the API. <br>

In [3]:
# Do not like when my md table is not aligned to the left
from IPython.core.display import HTML
table_css = 'table {align:left;display:block} '
HTML('<style>{}</style>'.format(table_css))

What you can use | ...depends on .py |     ...depends on .py| ... where it does that call to COD API (https://my.callofduty.com/api.papi-client/.)
:-------------|:--------------|:------------------|:------------------------------------
client.GetPlayer|*returns Player*||
client.SearchPlayers|http.SearchPlayer||crm/cod/v2/platform/platform/username/username/search
client.GetPlayerProfile|http.GetPlayerProfile||stats/cod/v1/title/title/platform/platform/gamer/username/profile/type/mode
client.GetPlayerMatches   -- if user platform = Activision |http.GetPlayerMatchesDetailed||crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime/details?limit=limit
client.GetPlayerMatches   -- if user platform = Bnet etc.. |http.GetPlayerMatches||crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime?limit=limit
client.GetPlayerMatchesSummary|http.GetPlayerMatchesDetailed||crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime/details?limit=limit
client.GetMatchDetails|http.GetMatch||ce/v1/title/title/platform/platform/match/matchId/matchMapEvents
client.GetMatch|*returns Match*||
client.GetFullMatch|http.GetFullMatch||crm/cod/v2/title/title/platform/platform/fullMatch/mode/matchId/language
client.GetLeaderboard|http.GetLaderboard||leaderboards/v2/title/title/platform/platform/time/timeFrame/type/gameType/mode/gameMode/page/page
*The preferred matches endpoint does not support Activision (uno) platform*|*should use (cf. postman):*||crm/cod/v2/title/title/platform/platform/uno/username/matches/mode/start/startTime/end/endTime/details?limit=limit
Leaderboad & getMatch endpoints matchMapEvents is for multiplayer only (no Warzone)*|||
*Others: GetMyFriends etc*|||
---|---||
player.profile |client.GetPlayerProfile|http.GetPlayerProfile|stats/cod/v1/title/title/platform/platform/gamer/username/profile/type/mode
player.matchesSummary|client.GetPlayerMatchesSummary|http.GetPlayerMatchesDetailed|crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/startTime/start/end/endTime/details?limit=limit
player.matches |client.GetPlayerMatches|http.GetPlayerMatches|crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/startTime/start/end/endTime}?limit=limit
*+ player.loadouts, player.loadoutUnlocks...*|||
---|---||
match.teams|client.GetMatchTeams||ce/v1/title/title/platform/platform/match/matchId/matchMapEvents
match.details|client.GetMatchDetails||ce/v1/title/title/platform/platform/match/matchId/matchMapEvents
*Endpoint matchMapEvents works for multiplayer only (no Warzone)*|||

# Private routes

Not our focus here but once logged in, you have access to private routes related to you own account (only) e.g. your friends' profiles (`client.GetMyFriends()`) and activity, account search visibility, used platform (e.g. Battlenet), identifiers linked to your Activision account etc. <br>
Cf. the test.py where ethanC have listed all the methods available in his client.

In [4]:
# For instance the .GetMyFriends() method, build the private endpoint to retrieve your friends statuses, using the authenticated client (personal credendials & associated gamertag).
friends = await client.GetMyFriends()
for friend in friends:
    print(f"{friend.username}, Online: {friend.online}")

chrissou#9246578, Online: False
Marmiton#4932812, Online: False
Moinolol#4713832, Online: False
nicoyzovitch#7591470, Online: False
ninjawariorbob#7568880, Online: False
Confetti_Seeker#1916728, Online: False


In [5]:
# Here, the client nicely returns a list of dict w. friends' info (our client's code exploration hints the COD API usually returns json or txt)
pprint(friends[0].__dict__)

{'_client': <callofduty.client.Client object at 0x7f9c8c725af0>,
 'accountId': '1722124035977126995',
 'avatarUrl': None,
 'identities': [],
 'online': False,
 'platform': <Platform.Activision: 'uno'>,
 'username': 'chrissou#9246578'}


# Public routes

Routes you can access without authentification. Mostly the leaderboards for COD classic ultiplayer modes (MW, BO4), as well as maps & modes available for multiplayer.<br>

### Leaderboards

Global ranking of players by their score, kills, kills deaths (kd) ratios etc.<br>
Searched a lot and in every manner possible but Warzone leaderboard (you can see in in-game :-p) endpoint is protected/out of my reach.<br>
Still, an ex. on how to get the leaderboard from the "Cyber" mode in COD Modern Warfare. The client return a leaderboard object, with entries (players) you can also dive into :

method | parameters | endpoint
:------|:-----------|---------------------------------------------------
client.GetPlayer|title, platform, --optional : gameType, gameMode, timeframes, page|leaderboards/v2/title/title/platform/platform/time/timeFrame/type/gameType/mode/gameMode/page/page

In [6]:
leaderboard = await client.GetLeaderboard(title=Title.ModernWarfare, platform=Platform.BattleNet, gameType=GameType.Core, gameMode="cyber", timeFrame=TimeFrame.AllTime, page=1)
pprint(leaderboard.__dict__, depth=1)
print('\n entries:')
for entry in leaderboard.entries[:3]:
    print(f"{entry.rank}: {entry.username} ({entry.platform.name})")

{'_client': <callofduty.client.Client object at 0x7f9c8c725af0>,
 'columns': [...],
 'entries': [...],
 'gameMode': 'cyber',
 'gameType': <GameType.Core: 'core'>,
 'page': 1,
 'pages': 407731,
 'platform': <Platform.BattleNet: 'battle'>,
 'timeFrame': <TimeFrame.AllTime: 'alltime'>,
 'title': <Title.ModernWarfare: 'mw'>}

 entries:
1: 小赵同学#1148917 (BattleNet)
2: BrattySis#6834874 (BattleNet)
3: RNYNN#6664890 (BattleNet)


In the client other methods that access public routes are available such as `.GetPlayerLeaderboard()` (returns the leadeboard'page for a particular user; no Warzone) and `.GetFullMatch()` (Wz compatible)

<a id='go_match'></a>

#### Match details : match > players stats in a given MatchId

Get detailed stats about a match given a match ID, (moderwarfare/multiplayer or modernware/Warzone etc.) <br>

method | parameters | endpoint
:------|:-----------|---------------------------------------------------
client.GetFullMatch|platform, title, mode, matchId, --default : language.English |crm/cod/v2/title/title/platform/platform/fullMatch/mode/matchId/language

In [7]:
# w. matchID taken from the Postman example. Later we will see how we can retrieve our own MatchIDs. 
# In this case a battle royale (Warzone) game with 145 players organizezd in teams of 4 (quads).
match = await client.GetFullMatch(Platform.Activision, Title.ModernWarfare, Mode.Warzone, matchId=18041330435098084962)

for i, dict_ in enumerate(match['allPlayers'][60:64]):
    print(i+1, dict_['player']['username'], dict_['player']['uno'])

1 exectiger 8250782718598582798
2 havengrave 16542688049064972663
3 YORKIE BG 9918567729791500373
4 gentil_renard 2621859779580650696


Returns a dict with a list of dict, every dict being a player-and-his-stats (here 145).<br>
In this match : 145 players, organized in teams of 4 ('br squad').<br>
Our selected player had 0 kills (playerStats.kills), 2 deathes (playerStats.deaths) and was moving 87% of the time (.percentTimeMoving) and the whole team ranked 31 (.teamPlacement)

In [8]:
# One given player (63th = me) returned stats among the 145 (note: should be 37 teams x 4 players = 148 players initially ?).
pprint(match['allPlayers'][63], depth=3)

{'draw': False,
 'duration': 1581000,
 'gameType': 'wz',
 'map': 'mp_don4',
 'matchID': '18041330435098084962',
 'mode': 'br_brtrios',
 'player': {'awards': {},
            'brMissionStats': {'missionStatsByType': {},
                               'missionsComplete': 0,
                               'totalMissionWeaponXpEarned': 0.0,
                               'totalMissionXpEarned': 0.0},
            'clantag': 'lkf :',
            'loadout': [{...}],
            'rank': 54.0,
            'team': 'team_twenty_three',
            'uno': '2621859779580650696',
            'username': 'gentil_renard'},
 'playerCount': 147,
 'playerStats': {'assists': 0.0,
                 'bonusXp': 0.0,
                 'challengeXp': 0.0,
                 'damageDone': 1972.0,
                 'damageTaken': 338.0,
                 'deaths': 2.0,
                 'distanceTraveled': 259886.23,
                 'executions': 0.0,
                 'gulagDeaths': 1.0,
                 'gulagKills': 

# Protected routes

Authentification is mandatory to access those. Good thing is that you can retrieve data for other players (w. visibility setting turned ON)

## Player search

One can play Warzone through PlayStation, PC (BattleNet) or Xbox (also, cross play), hence the username being tied to a platform when searching.<br>
Activision allows to change its own in-game username once in a while (3 months I believe). <br>
Players can share the same name, they differentiate with ending numbers (6 digits for Activision, 4 for Bnet). Max number of players returbed by the COD API is 20.<br>
The client return a list of `player` objects

method | parameters | endpoint
:------|:-----------|---------------------------------------------------
client.SearchPlayers|platform, username|crm/cod/v2/platform/platform/username/username/search

In [9]:
# For instance, my in-game --changed, username is gentil_renard, I can retrieve it (gentil_renard#3391079) with platform = Activision (translates into 'Uno' when the client builds the route)
results = await client.SearchPlayers(Platform.Activision, "gentil_renard")
for player in results:
    print(f"{player.username} ({player.platform.name})")

# but though I'm playing via Bnet, can't retrieve if I set platform = Bnet
results = await client.SearchPlayers(Platform.BattleNet, "gentil_renard")
for player in results:
    print(f"{player.username} ({player.platform.name})")

# Only works if I use my Bnet gamertag
results = await client.SearchPlayers(Platform.BattleNet, "AMADEVS#1689")
for player in results:
    print(f"{player.username} ({player.platform.name})")   

gentil_renard#3391079 (Activision)
Amadevs#1689 (BattleNet)


In [10]:
# A friend of mine uses a PlayStation
results = await client.SearchPlayers(Platform.PlayStation, "Nicoyzovitch")
for player in results:
    print(f"{player.username} ({player.platform.name})")

# Can also retrieve his name via Activision (nicoyzovitch#7591470) has he never changed his name.
results = await client.SearchPlayers(Platform.Activision, "Nicoyzovitch")
for player in results:
    print(f"{player.username} ({player.platform.name})")

nicoyzovitch (PlayStation)
nicoyzovitch#7591470 (Activision)


In [11]:
# Striking example with 'Huskerrs' (a popular pro player) wannabes . 
# Good thing Activision has an authenticity stamp you can retrieve with player name and phrase (cf. .authenticityStamp in the client)
res = []
for platform in [Platform.Activision, Platform.BattleNet]:
    res.extend(await client.SearchPlayers(platform, "HusKerrs"))

for player in res[:25]:
    print(f"{player.username} ({player.platform.name})")

Huskerrs (Activision)
HusKerrs#1009786 (Activision)
HusKerrs#1088477 (Activision)
HusKerrs#3209982 (Activision)
HusKerrs#4249229 (Activision)
HusKerrs#4780912 (Activision)
HusKerrs#5139476 (Activision)
HusKerrs#7232956 (Activision)
HusKerrs#7631054 (Activision)
HusKerrs#8490490 (Activision)
HusKerrs#8638305 (Activision)
HusKerrs#8653257 (Activision)
HusKerrs#9624907 (Activision)
HusKerrs#9783265 (Activision)
Huskerrs#2032932 (Activision)
Huskerrs#2058640 (Activision)
Huskerrs#3542853 (Activision)
Huskerrs#7010480 (Activision)
Huskerrs#8797872 (Activision)
Huskerrs#9357694 (Activision)
huskerrs#6821860 (Activision)
HusKerrs#1216 (BattleNet)
HusKerrs#1343 (BattleNet)
HusKerrs#1601 (BattleNet)
HusKerrs#1604 (BattleNet)


## Player profile

### Profile  / client methods

Two ways to retrieve player's profile in the client (same endpoint) : client.GetPlayerProfile or player.profile <br>

##### Profile using client.GetPlayerProfile

method | parameters | endpoint
:------|:-----------|---------------------------------------------------
client.GetPlayerProfile|platform, username, title, mode|stats/cod/v1/title/title/platform/platform/gamer/username/profile/type/mode

In [12]:
profile_using_client = await client.GetPlayerProfile(Platform.BattleNet, "AMADEVS#1689", Title.ModernWarfare, Mode.Warzone)
pprint(profile_using_client, depth=2)

{'engagement': None,
 'level': 14.0,
 'levelXpGained': 39309.0,
 'levelXpRemainder': 3591.0,
 'lifetime': {'accoladeData': {...},
              'all': {...},
              'itemData': {...},
              'map': {},
              'mode': {...},
              'scorestreakData': {...}},
 'maxLevel': 1.0,
 'maxPrestige': 0.0,
 'p': 0.0,
 'paragonId': 0.0,
 'paragonRank': 0.0,
 'platform': 'battle',
 'prestige': 23.0,
 'prestigeId': 0.0,
 's': 0.0,
 'title': 'mw',
 'totalXp': 1325315.0,
 'type': 'wz',
 'username': 'AMADEVS#1689',
 'weekly': {'all': {...}, 'map': {}, 'mode': {...}}}


##### Profile using player.profile

method | parameters | endpoint
:------|:-----------|---------------------------------------------------
client.GetPlayer|platform, username| returns Player Object -->
Player.profile|title, mode|-->
client.GetPlayerProfile|platform, username, title, mode|stats/cod/v1/title/title/platform/platform/gamer/username/profile/type/mode

In [13]:
# Getting player object first, as defined in player.py
player = await client.GetPlayer(Platform.BattleNet, "AMADEVS#1689")
print(f"{player.username} ({player.platform.name})")

# then, calling the .profile method
profile_using_player = await player.profile(Title.ModernWarfare, Mode.Warzone)
pprint(profile_using_player, depth=2)

AMADEVS#1689 (BattleNet)
{'engagement': None,
 'level': 14.0,
 'levelXpGained': 39309.0,
 'levelXpRemainder': 3591.0,
 'lifetime': {'accoladeData': {...},
              'all': {...},
              'itemData': {...},
              'map': {},
              'mode': {...},
              'scorestreakData': {...}},
 'maxLevel': 1.0,
 'maxPrestige': 0.0,
 'p': 0.0,
 'paragonId': 0.0,
 'paragonRank': 0.0,
 'platform': 'battle',
 'prestige': 23.0,
 'prestigeId': 0.0,
 's': 0.0,
 'title': 'mw',
 'totalXp': 1325315.0,
 'type': 'wz',
 'username': 'AMADEVS#1689',
 'weekly': {'all': {...}, 'map': {}, 'mode': {...}}}


### Profile / output from COD API (Warzone oriented)

If mode was set to Multiplayer, data would be globally similar but stats related to Modern Warfare game / multiplayers modes & maps instead of Warzone.<br>
Some stats may still be shared whether or not you are using mode multiplayer instead of Warzone (e.g. XpGained, and probably a lot more ?).<br>
Also, even if selected mode is Warzone, the API still appears to send Modern Warfare Multiplayers modes stats, as seen in result.lifetime.mode. <br>
Took me too much time already to figure out what does what exactly, and still not sure about a lot of things. The following tables will helps, hopefully.

#### Overview : lifetime vs. weekly

The client returns a nested dict : the key "data" from json sent by COD API. No means to change timespan Lifetime or Weekly when calling.<br>
`result['lifetime']` and `result['weekly']` can be further explored. They have similar keys (**all**, **mode**, **map**), except Lifetime having 3 more (**itemData**, **scorestreakData**, **accoladeData**). <br>

result [ key ]|key [ subkey ]| what ?
:-------------|:--------------|:------------------
--result . username||
--result . level||
--result . *a few others*...||
--result . **lifetime**||
|. all|*lifetime stats, no matter the map/mode you're playing (sum of all modes)*
|. mode|*lifestime stats, given a specified mode (br, br_dmz, br_all, arena, koth..).*
|. map|*empty for me, not sure exactly why, tested multiple players with different COD usages*
|. itemData|*additional key for 'lifetime' level --aka not available for weekly. Lifetime stats related to weapon & equipment*
|. scorestreakData|*additional key for 'lifetime' level. Lifetime stats re. 'special' equipment : uav, airstrikes etc*
|. accoladeData|*additional key for 'lifetime' level. Even more diverse and surprising lifetime stats*
---result . **weekly** |<br>
|. all|*weekly stats, no matter the map/mode you're playing (sum of all modes)*
|. mode|*weekly stats, given a specified mode (br, br_dmz, br_all, arena, koth..).*
|. map|*empty*
--result . *a few others*...<br>

#### A note about other keys

|--result. (...)  <br>
|--result . username <br>
|--result . level <br>
|--result . prestige <br>
|--result . levelXpGained...<br>
|--result . platform...<br>
|--result . *insert any here : engagement, prestige, maxLevel, totalXp, type...*

`result['username']`, `result['platform']`, `result['level']`) : given username & associated platform (you're searching a given username on the corresponding platform), level of the Player (1 to 1000) <br>
`result['level'])` & `result['levelXpGained']` : are shared between Modern Warfare multiplayer modes & Warzone. Can't remember if `['prestige']` depends on Level or challenges that has to be done in game.<br>

#### Focus on lifetime stats

result . lifetime | result . lifetime . x| result . lifetime . x . y . z|what ?
:-------------|:--------------|-------------|:-------
--result . *a few others*...|||
--result . **lifetime** ||||
|. all||*unique key is 'properties' with several lifetime stats attached, lifetime & all modes alltogether*
||. properties . accuracy|*lifetime shots accuracy (head or all ?)*
||. properties . wins|*lifetime number of wins, all modes (or br + resurgence ? not sure)*
||. properties . gamesPlayed|*lifetime n of games played, all modes*
||. properties . kdRatio|*lifetime kills/deaths ratio*
||. properties . *a lot of others :* | *bestKillStreak, headshots, bestKD, scorePerGame..., way more details than in per-mode-stats*
|. mode -||*lifestime stats, per mode (warzone : br, br_dmz, br_all. multiplayer : arena, koth..)* 
|||*mw mp modes returned even if mode set to wz*
||. **br** . properties . deaths|total n of deaths for **battle royale** mode only (solos + duos + trios + quads).*
||. br . properties . kills|*total n of kills ...*
||. br . properties . wins|*total n of wins ...*
||. br . properties . *others :* |*exhaustive list : cash, contracts, downs, gamesPlayed, kdRatio, objTime,* 
|||*revives, score, scorePerMinute, timePlayed, topTen, topFive, topTwentyFive*
||. **br_all** . properties . deaths|*total n of deaths for **battle royale + plunder + rebirth** ...*.
||. **br_dmz** . properties . deaths|*total n of deaths for **plunder**. Not sure if Rebirth included. I think not.*
||. *br_dmz . properties . *others*| *see exhaustive items listed above*
|. map
|. itemData
|. scorestreakData
|. accoladeData
--result . weekly |||
--result . *a few others*...|||

Among others, `result['lifetime']['scorestreakData']`, related to how many uav, airstrike, sentry guns... were used, might return the corresponding n of kills for each, but not sure. <br>
In `result['lifetime']['accoladeData']` , lifestats such as timeWatchingKillcams, timeWatchingKillcams, comebackKills... though not sure whether they are fully accurate or updated. E.g deathsFromBehind is 5 for me whereas I'm sure died a lot more that way :D

<a id='go_profile_weekly'></a>

#### Focus on weekly stats

`result['weekly']` does not have some keys yet available in `result['lifetime']` (itemData**, scorestreakData, accoladeData) ; meaning that you won't have weekly stats about weapons used, 'kill streaks' or other listed above.<br>
`result['weekly']['mode']` granularity (br_all, br_brduos, br_trios, br_brquads, br_dmz_plndtrios...) is different -- as more precise, from `result['lifetime']['mode']` (br, br_all, br_dmz)<br>
`result['weekly']['mode']` has also more detailed stats (items), for 'all' and each mode (e.g objectiveBrDownEnemyCircle1, objectiveBrDownEnemyCircle2, wallBangs...) than in `result['lifetime']

result . weekly | result . weekly . x| result . weekly . x . y . z|what ?
:-------------|:--------------|-------------|:-------
--result . *a few others*...|||
--result . lifetime ||||
--result . **weekly** ||||
|all||*unique key is 'properties' with several weekly stats attached. Data is **all modes** put together (br duos + plunder + rebirth...)* <br>
|| . properties . assists|*weekly n of kill assists*
|| . properties . deaths|*weekly n of deaths, all modes (or br + resurgence ? not sure)*
|| . properties . matchesPlayed|*weekly n of games played*
|| . properties . kdRatio|*weekly kills/deaths ratio*
|| . properties . *a lot of others : executions, objectiveBrDownEnnemyCircle1, timePlayed...*
|mode||*weekly stats, **per mode** (br, plunder, rebirth...)*<br>
|| . **br_all** . properties . assists|*weekly n of assists for **all modes** (solos + duos + trios + quads). Thus, **same items as weekly.all***<br>
|| . **br_solos** or **br_brduos** or **br_trios** or **br_quads** . properties . assists|*weekly n of assists for selected br mode.*
|| . **br_dmz_plndtrios** or **br_rebirth_rebirth_rex** or (...) . properties . assists|*weekly n of assists for selected Warzone multiplayer mode.*
|| . insert_mode_here . properties . *a lot of others : executions, objectiveBrDownEnnemyCircle1, timePlayed...*
|map
--result . *a few others*...|||

## Matches

Retrieve Player's last Matchs IDs.<br>
If you remember well, you can then explore them with [Match Details](#go_match)

##### Matches using client.GetPlayerMatches

method | parameters | endpoint
:------|:-----------|---------------------------------------------------
client.GetPlayerMatches|platform, title, mode, --default: startTimestamp, endTimestamp, limit|crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/startTime/start/end/endTime?limit=limit

notes :||<br>
:------|:-----------|---------------------------------------------------
startTimestamp : *useless, default is 0* | endTimestamp : *UTC timestamp in milliseconds* |limit : *client default set to 10, API returns max 20*

In [14]:
# Endpoint if platform == Activision : crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTimestamp/end/endTimestamp/details?limit=limit through method http.GetPlayerMatchesDetailed
matches_using_client = await client.GetPlayerMatches(Platform.Activision, "gentil_renard#3391079", Title.ModernWarfare, Mode.Warzone, endTimestamp=1614207600000, limit=2)
for match in matches_using_client:
    pprint(match.__dict__)

{'_client': <callofduty.client.Client object at 0x7f9c8c725af0>,
 'id': 11036841995021514111,
 'platform': <Platform.Activision: 'uno'>,
 'title': <Title.ModernWarfare: 'mw'>}
{'_client': <callofduty.client.Client object at 0x7f9c8c725af0>,
 'id': 15160474422724347965,
 'platform': <Platform.Activision: 'uno'>,
 'title': <Title.ModernWarfare: 'mw'>}


##### Matches using player.matches

method | parameters | endpoint
:------|:-----------|---------------------------------------------------
client.GetPlayer|platform, username| returns Player Object -->
Player.matches|title, mode|-->
client.GetPlayerMatches|platform, title, mode, --default: startTimestamp, endTimestamp, limit|crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/startTimestamp/start/end/endTimestamp?limit=limit

In [15]:
# Getting player object first, as defined in player.py
player = await client.GetPlayer(Platform.BattleNet, "AMADEVS#1689")
print(f"{player.username} ({player.platform.name})")

# then, calling the .matches method
# Endpoint if platform = Activision              : crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime/details?limit=limit through method http.GetPlayerMatchesDetailed
# Returns a [n match objects]
matches_using_player = await player.matches(Title.ModernWarfare, Mode.Warzone, limit =2)
for match in matches_using_player:
    pprint(match.__dict__)

AMADEVS#1689 (BattleNet)
{'_client': <callofduty.client.Client object at 0x7f9c8c725af0>,
 'id': 655340174593516288,
 'platform': <Platform.BattleNet: 'battle'>,
 'title': <Title.ModernWarfare: 'mw'>}
{'_client': <callofduty.client.Client object at 0x7f9c8c725af0>,
 'id': 3288905818470012806,
 'platform': <Platform.BattleNet: 'battle'>,
 'title': <Title.ModernWarfare: 'mw'>}


## Matches Summary

### Matches Summary / methods

##### Summary using client.GetPlayesMatchesSummary

method | parameters | endpoint
:------|:-----------|---------------------------------------------------
client.GetPlayerMatchesSummary|platform, title, mode, --default: startTimestamp, endTimestamp, limit|crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTimestamp/end/endTimestamp/details?limit=limit

notes :||<br>
:------|:-----------|---------------------------------------------------
startTimestamp : *useless param, default is always 0* | endTimestamp : *UTC timestamp in milliseconds* |limit : *client default set to 10, API returns max 20*

In [16]:
# Returns consolidated stats for n last matches, all + per type  br, plunder ...
summary = await client.GetPlayerMatchesSummary(Platform.BattleNet, "AMADEVS#1689", Title.ModernWarfare, Mode.Warzone, limit=15)
pprint(summary)

{'all': {'assists': 17.0,
         'avgLifeTime': 239.06666666666666,
         'damageDone': 31765.0,
         'damageTaken': 15079.0,
         'deaths': 120.0,
         'distanceTraveled': 5470203.6099999985,
         'executions': 3.0,
         'gulagDeaths': 7.0,
         'gulagKills': 1.0,
         'headshotPercentage': 0.2647058823529412,
         'headshots': 27.0,
         'kdRatio': 0.85,
         'kills': 102.0,
         'killsPerGame': 6.8,
         'matchesPlayed': 15.0,
         'nearmisses': 0.0,
         'objectiveBrCacheOpen': 30.0,
         'objectiveBrDownEnemyCircle1': 10.0,
         'objectiveBrDownEnemyCircle2': 1.0,
         'objectiveBrDownEnemyCircle4': 2.0,
         'objectiveBrDownEnemyCircle5': 1.0,
         'objectiveBrKioskBuy': 6.0,
         'objectiveBrMissionPickupTablet': 4.0,
         'objectiveDestroyedEquipment': 3.0,
         'objectiveLastStandKill': 42.0,
         'objectiveReviver': 3.0,
         'objectiveTeamWiped': 50.0,
         'objectiveTrop

##### Summary using player.matchesSummary

method | parameters | endpoint
:------|:-----------|---------------------------------------------------
client.GetPlayer|platform, username| returns Player Object -->
Player.matchesSummary|title, mode|-->
client.GetPlayerMatchesSummary|platform, title, mode, --default: startTimestamp, endTimestamp, limit|crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTimestamp/end/endTimestamp/details?limit=limit

In [17]:
# Getting player object first, as defined in player.py
player = await client.GetPlayer(Platform.BattleNet, "AMADEVS#1689")
print(f"{player.username} ({player.platform.name})")

# then, calling the .matchesSummary method
# Returns stats for n last matches, all + per type  br, plunder ...
summary = await player.matchesSummary(Title.ModernWarfare, Mode.Warzone, limit=15) # does not work if I use player = await client.GetPlayer(Platform.Activision, "gentil_renard") cf. Postman Warzone by Uno ID ?
pprint(summary, depth=1)

AMADEVS#1689 (BattleNet)
{'all': {...}, 'br_brtrios': {...}, 'br_dmz_plunquad': {...}}


### Matches Summary / output from COD API (Warzone oriented)

The returned data share almost the same hierarchy and same stats as what's returned for [Player Profile, Weekly entry.](#go_profile_weekly)

# Client "enhancements"

### Handle a "new" API endpoint

As listed on Postman, COD API has an another endpoint (see code below) currently not used by the client. I was curious whether or not it would give us anything different. <br>
Turned out the additional method return n matches stats (summary + details per match), similar to what you would get with client.GetPlayersMatchesDetailed().<br>
The only difference is that it takes UnoID as a parameter e.g. 12309926, different from Activision User ID e.g 'gentil_renard#3391079'. Additionally something you cannot do with other routes.<br>
You can retrieve this UnoID, for every player, when you call for the GetFullMatch method for instance. <br>
I learned only afterwards, that before January 2021 Activision would allow to use this endpoint/**UnoID** to get stats. it **does not work** anymore, **except for your own account** / your friends.

#### Add additional methods at runtime --aka w/o modifying the package

In [18]:
# we're adding additional methods in the Call of Duty .py client, without modifying the package.
# Import the Class we want to modify, without touching the client's package

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request
import urllib.parse

# define additonal method in callofduty.http HTTP Class
# Endpoint extracted from Postman, currently not supported by the client (but turned out not to be mandatory tbh <3)

async def GetWithUnoID(self, platform, username, title, mode, limit, startTimestamp, endTimeStamp):
    return await self.Send(
        Request(
            "GET",
            f"api/papi-client/crm/cod/v2/title/{title}/platform/{platform}/uno/{urllib.parse.quote(username)}/matches/{mode}/start/{startTimestamp}/end/{endTimeStamp}/details?limit={limit}",
        )
    )
# define additional method in callofduty.client Client Class
# We chose not to differentiate between 'summary'and 'matches" entries when returning the results --as done in the the client (aka we return them both).

async def GetMatchesWithUnoID(self, platform: Platform, username, title: Title, mode: Mode, **kwargs):
    limit = kwargs.get("limit", 10)
    startTimestamp = kwargs.get("startTimestamp", 0)
    endTimestamp = kwargs.get("endTimestamp", 0)

    data = (
        await self.http.GetWithUnoID(
            platform.value,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp
        )
    )["data"]

    return data

# let's add our additional methods into respective Classes, at runtime

Client.GetMatchesWithUnoID = GetMatchesWithUnoID
HTTP.GetWithUnoID = GetWithUnoID

#### New endpoint : results

In [19]:
# Now we can use our new method as if it was in the package ^_^
matches_with_unoID = await client.GetMatchesWithUnoID(Platform.Activision, "2621859779580650696", Title.ModernWarfare, Mode.Warzone, limit=2)
pprint(matches_with_unoID, depth=3)

{'matches': [{'draw': False,
              'duration': 1078000,
              'gameType': 'wz',
              'map': 'mp_don4',
              'matchID': '655340174593516288',
              'mode': 'br_dmz_plunquad',
              'player': {...},
              'playerCount': 100,
              'playerStats': {...},
              'playlistName': None,
              'privateMatch': False,
              'rankedTeams': None,
              'teamCount': 46,
              'utcEndSeconds': 1633883514,
              'utcStartSeconds': 1633882436,
              'version': 1},
             {'draw': False,
              'duration': 872000,
              'gameType': 'wz',
              'map': 'mp_don4',
              'matchID': '3288905818470012806',
              'mode': 'br_dmz_plunquad',
              'player': {...},
              'playerCount': 101,
              'playerStats': {...},
              'playlistName': None,
              'privateMatch': False,
              'rankedTeams': None,
  

### Remove some client-built-in-filters to "fully" exploit COD API

Took me some time to figure that out --given 1. that Activision tends to be secretive on its API, and have been introducing some limitations over time 2. Do not know the initial motivations of client creator EthanC. but :<br>
Really wanted to be able to get Player's n last matches with their corresponding detailed stats attached to each entry. <br>
Currently the client exploit either the /matches/ endpoint through client.GetPlayerMatches : return the n last matches IDs<br>
or the /matches/details endpoint through client.GetPlayerMatchesDetailed : returns the aggregated stats of the n last matches (all matches, or with the details for each mode : plunder vs. battle_royale for instance)<br>
Turned out that COD API / matches/details endpoint returns both matches aggregated stats (summary) but also the stats I wanted initially, but this was filtered out inside aforementioned methods. <br>
Still thankful and very statisfied on how and what Ethan C python client provides, so we will add what we want without rewriting or modifying his great work.<br>

#### Copy and edit GetPlayerMatches GetPlayerMatchesDetailed

In [20]:
# This time we're adding additional methods in the Call of Duty .py client only as there is no need to modify the HTTP class that already contains the endpoint we want to use
# Import the Class we want to modify

import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request


# define two additional methods in callofduty.client Client Class

async def GetPlayerMatchesRaw(self, platform: Platform, username: str, title: Title, mode: Mode, **kwargs):

    limit: int = kwargs.get("limit", 10)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    data: dict = (
        await self.http.GetPlayerMatches(
            platform.value,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"] # was filtered out here

    return data


async def GetPlayerMatchesDetailedRaw(self, platform: Platform, username: str, title: Title, mode: Mode, **kwargs):

    limit: int = kwargs.get("limit", 10)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform.value,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"] # was filtered out here

    return data

# let's add our additional methods into Client Class, at runtime

Client.GetPlayerMatchesRaw = GetPlayerMatchesRaw
Client.GetPlayerMatchesDetailedRaw = GetPlayerMatchesDetailedRaw

#### Result : get Player's n last matches, with stats attached

Activision platform is not supported by the API. Works if we provide a BattleNet, Xbox, Playstation platform and corresponding username.<br>
Client is keeping history of matchID, platform, title; but we might also be interesting to make usage of timestamp, match type and map id (in the future :p)

In [21]:
# Now we can use our new methods as if it was in the package 
raw_matches_acti = await client.GetPlayerMatchesRaw(Platform.Activision, "gentil_renard#3391079", Title.ModernWarfare, Mode.Warzone, limit=5)
raw_matches_bnet = await client.GetPlayerMatchesRaw(Platform.BattleNet, "AMADEVS#1689", Title.ModernWarfare, Mode.Warzone, limit=5)
print(raw_matches_acti)
pprint(raw_matches_bnet)

[]
[{'map': '1418338122340459',
  'matchId': '655340174593516288',
  'platform': 'battle',
  'timestamp': 1633883514000,
  'title': 'mw',
  'type': '6870190091107131'},
 {'map': '1418338122340459',
  'matchId': '3288905818470012806',
  'platform': 'battle',
  'timestamp': 1633882202000,
  'title': 'mw',
  'type': '6870190091107131'},
 {'map': '1418338122340459',
  'matchId': '18041330435098084962',
  'platform': 'battle',
  'timestamp': 1633730237000,
  'title': 'mw',
  'type': '4171952442769456'},
 {'map': '1418338122340459',
  'matchId': '16730115467557733083',
  'platform': 'battle',
  'timestamp': 1633729503000,
  'title': 'mw',
  'type': '4171952442769456'},
 {'map': '1418338122340459',
  'matchId': '12695423864955715863',
  'platform': 'battle',
  'timestamp': 1633727856000,
  'title': 'mw',
  'type': '4171952442769456'}]


In [22]:
# raw_matches_datailes has indeed two keys : summary and matches (the one that is filtered out in the Client)
raw_matches_detailed = await client.GetPlayerMatchesDetailedRaw(Platform.BattleNet, "AMADEVS#1689", Title.ModernWarfare, Mode.Warzone, limit=5)
pprint(raw_matches_detailed['summary'])

{'all': {'assists': 8.0,
         'avgLifeTime': 270.7111111111111,
         'damageDone': 11183.0,
         'damageTaken': 5090.0,
         'deaths': 40.0,
         'distanceTraveled': 1741115.5899999999,
         'executions': 0.0,
         'gulagDeaths': 3.0,
         'gulagKills': 1.0,
         'headshotPercentage': 0.21212121212121213,
         'headshots': 7.0,
         'kdRatio': 0.825,
         'kills': 33.0,
         'killsPerGame': 6.6,
         'matchesPlayed': 5.0,
         'nearmisses': 0.0,
         'objectiveBrCacheOpen': 10.0,
         'objectiveBrDownEnemyCircle1': 7.0,
         'objectiveBrDownEnemyCircle2': 1.0,
         'objectiveBrDownEnemyCircle4': 1.0,
         'objectiveBrKioskBuy': 4.0,
         'objectiveBrMissionPickupTablet': 2.0,
         'objectiveDestroyedEquipment': 1.0,
         'objectiveLastStandKill': 12.0,
         'objectiveReviver': 1.0,
         'objectiveTeamWiped': 19.0,
         'score': 19325.0,
         'scorePerMinute': 95.18141520275817,
 

In [23]:
pprint(raw_matches_detailed['matches'][0], depth=4)

{'draw': False,
 'duration': 1078000,
 'gameType': 'wz',
 'map': 'mp_don4',
 'matchID': '655340174593516288',
 'mode': 'br_dmz_plunquad',
 'player': {'awards': {},
            'brMissionStats': {'missionStatsByType': {},
                               'missionsComplete': 0,
                               'totalMissionWeaponXpEarned': 0.0,
                               'totalMissionXpEarned': 0.0},
            'clantag': 'lkf :',
            'loadout': [{'extraPerks': [...],
                         'killstreaks': [...],
                         'lethal': {...},
                         'perks': [...],
                         'primaryWeapon': {...},
                         'secondaryWeapon': {...},
                         'tactical': {...}}],
            'rank': 54.0,
            'team': 'team_seventeen',
            'uno': '2621859779580650696',
            'username': 'gentil_renard'},
 'playerCount': 100,
 'playerStats': {'assists': 5.0,
                 'bonusXp': 0.0,
         